In [1]:
datadir = "../data/"
datafilename = "Certainty.xlsx"
datafilepath = datadir + datafilename

In [2]:
from itertools import combinations
import pandas as pd
import numpy as np

# Read in network information from sheet P0
# For now we only use the folloiwng columns in each sheet
cols_used = ['Link ID','ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','Link Length(miles)','# of lanes-A','Capacity-A (veh/h)',
            'auto volume(2010)-A','AADT(2010)-A','Speed(mph)-A','VMT-A']
df_p0= pd.read_excel(datafilepath, sheet_name='P0',usecols=cols_used).dropna(subset=['Link ID'])

# # For now we assume all occurrences of the same link in the same sheet share the same attribute values
# df_p0_unique = df_p0.drop_duplicates(subset=['Link ID']).dropna(subset=['Link ID', 'A X_COORD', 'A Y_COORD', 'B X_COORD', 'B Y_COORD'])


In [3]:
### Only Read in Project 3, 4 and 5
integers = [ 3, 4, 5]
def generate_combinations(integers):
    all_combinations = []
    # Loop through lengths from 1 to 6
    for length in range(1, 1+len(integers)):
        # Generate combinations of the current length
        comb = combinations(integers, length)
        # Convert each combination to a string, add "p", and add to the list
        all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
    return all_combinations
# Generate all combinations
sheet_names = generate_combinations(integers)

# Insert "P0" at the beginning of the list
sheet_names.insert(0, "P0")


In [4]:
##### Read in all 2^6 sheets of data ###


# ## Create a list of sheet names ##

# # List of the first 6 integers
# integers = [1, 2, 3, 4, 5, 6]

# # Function to generate combinations
# def generate_combinations(integers):
#     all_combinations = []
#     # Loop through lengths from 1 to 6
#     for length in range(1, 7):
#         # Generate combinations of the current length
#         comb = combinations(integers, length)
#         # Convert each combination to a string, add "p", and add to the list
#         all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
#     return all_combinations

# # Generate all combinations
# sheet_names = generate_combinations(integers)

# # Insert "P0" at the beginning of the list
# sheet_names.insert(0, "P0")



In [5]:
import torch
import torch.nn.functional as F 
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv
import random


### Data Preparation ###

######### Nodes ##############
# colums that contain info about nodes in the graph
node_info_cols = ['ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD']
df_nodes= pd.read_excel(datafilepath, sheet_name='P0',usecols=node_info_cols).dropna(subset=['ANODE'])
# unique nodes among ANODE
df_nodes_ANODE_unique = df_nodes.drop_duplicates(subset=['ANODE'])
# unique nodes among BNODE
df_nodes_BNODE_unique = df_nodes.drop_duplicates(subset=['BNODE'])

# index of unique nodes in ANODE
ANODE_ind = df_nodes_ANODE_unique.index
# X-coord and Y-coord of nodes listed as ANODES 
Anodes = np.array(df_p0.iloc[ANODE_ind-1].loc[:,['ANODE','A X_COORD','A Y_COORD']]).astype(float)

# index of unique nodes in BNODE
BNODE_ind = df_nodes_BNODE_unique.index
# X-coord and Y-coord of nodes listed as BNODES 
Bnodes = np.array(df_p0.iloc[BNODE_ind-1].loc[:,['BNODE','B X_COORD','B Y_COORD']]).astype(float)

# combine Anodes and Bnodes
nodes = np.concatenate((Anodes, Bnodes))
# combine the two index arrays into one, without repetition
nodes_pd = pd.DataFrame(nodes)
nodes_unique = nodes_pd.drop_duplicates(subset=[0])
# Reset the index
nodes_unique.reset_index(drop=True, inplace=True)

# convert nodes_unique to np array
nodes_unique_arr = np.array(nodes_unique)
x =  torch.from_numpy(nodes_unique_arr).to(dtype=torch.float)


######### Edge Index #########
edge_pairs = np.column_stack((df_p0['ANODE'],df_p0['BNODE']))
edge_index = []
for ii in range(len(edge_pairs)):
    edge_node_ind = [nodes_unique[nodes_unique[0] == edge_pairs[ii,0]].index[0], nodes_unique[nodes_unique[0] == edge_pairs[ii,1]].index[0]]
    # edge_index[:,ii]= edge_node_ind
    edge_index.append(edge_node_ind)
edge_index = torch.tensor(edge_index).t().contiguous() 



/home/duantu/soft/transportation_venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Normalize data to be in the range (0,1) with minmaxsclaer


In [ ]:
# Prepare the dataset 
dataset = []
for ii in range(len(sheet_names)):
    # read the correct sheet 
    df= pd.read_excel(datafilepath, sheet_name=sheet_names[ii],usecols=cols_used).dropna(subset=['Link ID'])
    
    # dataframe for edge attributes
    df_edge_attr= np.column_stack((df['# of lanes-A'],df['Capacity-A (veh/h)'], df['Speed(mph)-A'],df['auto volume(2010)-A']))
    edge_attr = torch.tensor(df_edge_attr, dtype=torch.float)
    
    edge_labels = torch.tensor(df['AADT(2010)-A'].to_numpy(),dtype=torch.float)
    new_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=edge_labels)
    dataset.append(new_data)

In [6]:

class EdgeLabelPredictor(torch.nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim):
        super(EdgeLabelPredictor, self).__init__()
        self.conv1 = TransformerConv(node_features, hidden_dim, edge_dim=edge_features)
        self.conv2 = TransformerConv(hidden_dim, hidden_dim, edge_dim=edge_features)
        # self.edge_fc = torch.nn.Linear(hidden_dim, 1)
        self.edge_fc = torch.nn.Linear(2 * hidden_dim+edge_features, 1)
    
    def forward(self, x, edge_index, edge_attr):
        # import pdb;pdb.set_trace()
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        edge_features = torch.cat([x[edge_index[0]], x[edge_index[1]], edge_attr],dim=1)
        edge_features = self.edge_fc(edge_features)
        return F.relu(edge_features)  # Ensure the output is positive

# Example data
node_features = 3
edge_features = 4
hidden_dim = 16

model = EdgeLabelPredictor(node_features, edge_features, hidden_dim)




In [7]:


# dataset = [data]
# Create a data loader (replace with your actual dataset)
loader = DataLoader(dataset, batch_size=1,shuffle=True)

# Training settings
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.MSELoss()

# Training loop
model.train()
for epoch in range(100):  # Adjust the number of epochs as needed
    # ii=1
    epoch_loss = 0.0
    for batch in loader:
    
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr).squeeze()
        # print('batch #',ii, batch.edge_attr)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        # ii = ii+1
        
    # print(f'Epoch {epoch + 1}, Absolute Loss: {format(loss.item(),".2e")}')
    print(f'Epoch {epoch + 1}, Absolute Loss: {format(epoch_loss/len(loader),".2e")}')
############ DEBUGGING FOR ALL ZERO GRADIENTS #######
# for epoch in range(10):  # Adjust the number of epochs as needed
#     epoch_loss = 0.0
#     for ii, batch in enumerate(loader, start=1):
#                 optimizer.zero_grad()
#                 out = model(batch.x, batch.edge_index, batch.edge_attr).squeeze()
    
#                 loss = criterion(out, batch.y)
#                 loss.backward()
                
#                 # Print gradients for debugging
#                 for name, param in model.named_parameters():
#                     if param.grad is not None:
#                         print(f"Gradients for {name}: {param.grad.norm().item():.2e}")
#                     else:
#                         print(f"No gradients for {name}")
#                 # for name, param in model.named_parameters():
                #     if param.requires_grad:
                #         print(f"{name}: {param.data[:5]}")      
    #             optimizer.step()
    
    #             epoch_loss += loss.item()
    #             print(f'Batch #{ii}, Batch Edge Attr: {batch.edge_attr}')
        
    # print(f'Epoch {epoch + 1}, Absolute Loss: {format(epoch_loss, ".2e")}')






# Model evaluation 
model.eval()

#     ### Model eval on 1 randomly selected sample ###
# with torch.no_grad():
#     # randomly shuffle the set of 8 samples and then use the first sample for model eval
#     random.shuffle(dataset)
#     edge_attr_eval = dataset[0].edge_attr 
#     out = model(x, edge_index, edge_attr_eval).squeeze()
#     mse = criterion(out, edge_labels.squeeze()).item()
#     print(f'Mean Squared Error on a randomly selected sample: {format(mse, ".2e")}')

    ### Model eval on the entire dataset of 8 samples ###
all_preds = []
all_labels = []
    
with torch.no_grad():
    for batch in loader:
        out = model(batch.x, batch.edge_index, batch.edge_attr).squeeze()
        all_preds.append(out)
        all_labels.append(batch.y)  

# Concatenate all predictions and labels
all_preds = torch.cat(all_preds, dim=0)
all_labels = torch.cat(all_labels, dim=0)

mse = criterion(all_preds, all_labels.squeeze()).item()
print(f'Mean Squared Error on the entire dataset: {format(mse, ".2e")}')

    

# Calculate Relative Error
edge_labels_tensor = criterion(torch.zeros(torch.Tensor.size(edge_labels)), edge_labels)
loss_relative = mse/edge_labels_tensor

print("\n Relative Loss = ", format(mse, '.2e'), " / ", format(edge_labels_tensor.item(),'.2e'), " = ", format(loss_relative.item(),'.2e'), "\n")
    

> /tmp/ipykernel_37611/3971059052.py(11)forward()
      9     def forward(self, x, edge_index, edge_attr):
     10         import pdb;pdb.set_trace()
---> 11         x = F.relu(self.conv1(x, edge_index, edge_attr))
     12         x = F.relu(self.conv2(x, edge_index, edge_attr))
     13         edge_features = torch.cat([x[edge_index[0]], x[edge_index[1]], edge_attr],dim=1)



ipdb>  x


tensor([[   8921.0000,  407624.1875, 4649654.5000],
        [   9137.0000,  409288.5938, 4649934.5000],
        [   9027.0000,  408444.0000, 4649861.0000],
        ...,
        [   5142.0000,  343164.3125, 4677479.0000],
        [   5114.0000,  338274.5000, 4679260.0000],
        [   8694.0000,  405621.8125, 4648465.5000]])


ipdb>  x.shape


torch.Size([642, 3])


...

ipdb>  n


--Return--
tensor([[ 316...eluBackward0>)
> /tmp/ipykernel_37611/3971059052.py(15)forward()
     13         edge_features = torch.cat([x[edge_index[0]], x[edge_index[1]], edge_attr],dim=1)
     14         edge_features = self.edge_fc(edge_features)
---> 15         return F.relu(edge_features)  # Ensure the output is positive
     16 
     17 # Example data



ipdb>  n


--Return--
tensor([[ 316...eluBackward0>)
> /home/duantu/soft/transportation_venv/lib/python3.12/site-packages/torch/nn/modules/module.py(1541)_call_impl()
   1539                 or _global_backward_pre_hooks or _global_backward_hooks
   1540                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1541             return forward_call(*args, **kwargs)
   1542 
   1543         try:



ipdb>  q


In [ ]:

print('The predicted AADT for P0: \n', all_preds[:650])

In [ ]:
len(loader)